Original notebook in my drive: https://colab.research.google.com/drive/1F99hX8ce-Q15G4-KJWQm51OB167JWMDx

In [1]:
Tacotron_model = "1uPL8GbAZ6uPmNbM0yXgood6gR_hYuUNx"

TACOTRON2_ID = Tacotron_model
HIFIGAN_ID = "1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW"

git_repo_url = 'https://github.com/NVIDIA/tacotron2.git'
d = 'https://drive.google.com/uc?id='

graph_width = 900
graph_height = 360


In [2]:
def plot_data(data, figsize=(int(graph_width/100), int(graph_height/100))):
            %matplotlib inline
            fig, axes = plt.subplots(1, len(data), figsize=figsize)
            for i in range(len(data)):
                axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                            interpolation='none', cmap='inferno')
            fig.canvas.draw()
            plt.show()
            
def ARPA(text, punctuation=r"!?,.;", EOS_Token=True):
            out = ''
            for word_ in text.split(" "):
                word=word_; end_chars = ''
                while any(elem in word for elem in punctuation) and len(word) > 1:
                    if word[-1] in punctuation: end_chars = word[-1] + end_chars; word = word[:-1]
                    else: break
                try:
                    word_arpa = thisdict[word.upper()]
                    word = "{" + str(word_arpa) + "}"
                except KeyError: pass
                out = (out + " " + word + end_chars).strip()
            if EOS_Token and out[-1] != ";": out += ";"
            return out

def get_hifigan(MODEL_ID):
    # Download HiFi-GAN
    hifigan_pretrained_model = 'hifimodel'
    #gdown.download(d+MODEL_ID, hifigan_pretrained_model, quiet=False)
    if not exists(hifigan_pretrained_model):
        raise Exception("HiFI-GAN model failed to download!")

    # Load HiFi-GAN
    conf = os.path.join("hifi-gan", "config_v1.json")
    with open(conf) as f:
        json_config = json.loads(f.read())
    h = AttrDict(json_config)
    torch.manual_seed(h.seed)
    hifigan = Generator(h).to(torch.device("cuda"))
    state_dict_g = torch.load(hifigan_pretrained_model, map_location=torch.device("cuda"))
    hifigan.load_state_dict(state_dict_g["generator"])
    hifigan.eval()
    hifigan.remove_weight_norm()
    return hifigan, h

def has_MMI(STATE_DICT):
            return any(True for x in STATE_DICT.keys() if "mi." in x)

def get_Tactron2(MODEL_ID):
    # Download Tacotron2
    tacotron2_pretrained_model = 'Belinda-122'
    #gdown.download(d+MODEL_ID, tacotron2_pretrained_model, quiet=False)
    if not exists(tacotron2_pretrained_model):
        raise Exception("Tacotron2 model failed to download!")
    # Load Tacotron2 and Config
    hparams = create_hparams()
    hparams.sampling_rate = 22050
    hparams.max_decoder_steps = 3000 # Max Duration
    hparams.gate_threshold = 0.25 # Model must be 25% sure the clip is over before ending generation
    model = Tacotron2(hparams)
    state_dict = torch.load(tacotron2_pretrained_model)['state_dict']
    if has_MMI(state_dict):
        raise Exception("ERROR: This notebook does not currently support MMI models.")
    model.load_state_dict(state_dict)
    _ = model.cuda().eval().half()
    return model, hparams

In [3]:
# Extra Info
#import wave
#from playsound import playsound
import sounddevice
def end_to_end_infer(text, pronounciation_dictionary, show_graphs):
    for i in [x for x in text.split("\n") if len(x)]:
        if not pronounciation_dictionary:
            if i[-1] != ";": i=i+";" 
        else: i = ARPA(i)
        with torch.no_grad(): # save VRAM by not including gradients
            sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
            sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
            mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
            if show_graphs:
                plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
                        alignments.float().data.cpu().numpy()[0].T))
            y_g_hat = hifigan(mel_outputs_postnet.float())
            audio = y_g_hat.squeeze()
            audio = audio * MAX_WAV_VALUE
            
            return audio.cpu().numpy().astype("int16"), hparams.sampling_rate
                        
            

In [6]:
import time
print("Setting up, please wait.\n")
t0 = time.time()
from tqdm.notebook import tqdm
with tqdm(total=5, leave=False) as pbar:
    #%tensorflow_version 1.x
    import os
    from os.path import exists, join, basename, splitext
    #!pip install gdown        #Not needed anymore
    project_name = splitext(basename(git_repo_url))[0]
    if not exists(project_name):
        # clone and install
        !git clone -q --recursive {git_repo_url}
        !git clone -q --recursive https://github.com/SortAnon/hifi-gan
    pbar.update(1) # downloaded TT2 and HiFi-GAN
    import sys
    sys.path.append('hifi-gan')
    sys.path.append(project_name)
    import time
    import matplotlib
    import matplotlib.pylab as plt
    #import gdown


    %matplotlib inline
    import IPython.display as ipd
    import numpy as np
    import torch
    import json
    from hparams import create_hparams
    from model import Tacotron2
    from layers import TacotronSTFT
    from audio_processing import griffin_lim
    from text import text_to_sequence
    from env import AttrDict
    from meldataset import MAX_WAV_VALUE
    from models import Generator

    pbar.update(1) # initialized Dependancies

    # Setup Pronounciation Dictionary
    thisdict = {}
    for line in reversed((open('merged.dict.txt', "r").read()).splitlines()):
        thisdict[(line.split(" ",1))[0]] = (line.split(" ",1))[1].strip()

    pbar.update(1) # Downloaded and Set up Pronounciation Dictionary        

    hifigan, h = get_hifigan(HIFIGAN_ID)
    pbar.update(1) # Downloaded and Set up HiFi-GAN        

    model, hparams = get_Tactron2(TACOTRON2_ID)
    previous_tt2_id = TACOTRON2_ID

    pbar.update(1) # Downloaded and Set up Tacotron2


from IPython.display import clear_output
clear_output()
print("Total initializing time: ", time.time() - t0)

Total initializing time:  1.7928369045257568


Functions

Initialization (def need to improve this one)

In [16]:
if previous_tt2_id != TACOTRON2_ID:
    print("Updating Models")
    model, hparams = get_Tactron2(TACOTRON2_ID)
    hifigan, h = get_hifigan(HIFIGAN_ID)
    previous_tt2_id = TACOTRON2_ID

pronounciation_dictionary = False #@param {type:"boolean"}
# disables automatic ARPAbet conversion, useful for inputting your own ARPAbet pronounciations or just for testing
show_graphs = False #@param {type:"boolean"}
max_duration = 25 #this does nothing
model.decoder.max_decoder_steps = 100000 #@param {type:"integer"}
stop_threshold = 0.324 #@param {type:"number"}
model.decoder.gate_threshold = stop_threshold


print(f"Current Config:\npronounciation_dictionary: {pronounciation_dictionary}\nshow_graphs: {show_graphs}\nmax_duration (in seconds): {max_duration}\nstop_threshold: {stop_threshold}\n\n")

Current Config:
pronounciation_dictionary: False
show_graphs: False
max_duration (in seconds): 25
stop_threshold: 0.324




In [17]:
t0 = time.time()
line = "Una nueva frase larga que quiero ver cuanto se tarda."
audio, rate = end_to_end_infer(line, False, False)
t1 = time.time()
print("Start playing..")
sounddevice.play(audio, samplerate=rate)
sounddevice.wait()
t2 = time.time()
print("Time to infer", t1 - t0, sep=":")
print("Time to speak", t2 - t0, sep=":")
print("Audio length", t2 - t1, sep=":")
print((t1-t0)/(t2 - t0))

Start playing..
Time to infer:0.6579978466033936
Time to speak:4.103370904922485
Audio length:3.445373058319092
0.16035543991747037


This is the one that executes in a loop

In [10]:
print("Enter/Paste your text.")
contents = []
while True:
    try:
        print("-"*50)
        line = input()
        if line == "":
            continue
        audio, rate = end_to_end_infer(line, pronounciation_dictionary, show_graphs)
        display(ipd.Audio(audio, rate=rate))
    except EOFError:
        break
    except KeyboardInterrupt:
        print("Stopping...")
        break

Enter/Paste your text.
--------------------------------------------------


 Córdoba, también conocida como Lomas de Huilango (náhuatl Huilango) es una ciudad mexicana, ubicada en el centro del Estado de Veracruz, importante punto del antiguo corredor comercial de la Ciudad de México al Puerto de Veracruz. Está enclavada a la orilla de la cordillera montañosa de la Sierra Madre Oriental y en la parte sur de la región montañosa del estado de Veracruz.


Warning! Reached max decoder steps


--------------------------------------------------


 Está enclavada a la orilla de la cordillera montañosa de la Sierra Madre Oriental y en la parte sur de la región montañosa del estado de Veracruz.


--------------------------------------------------


 Está enclavada a la orilla de la cordillera montaniosa de la Sierra Madre Oriental y en la parte sur de la región montaniosa del estado de Veracruz.


--------------------------------------------------
Stopping...
